In [1]:
import urllib.parse
import numpy as np
import pandas as pd
import random as rd

domain = "51.91.251.0"
port = 3000
host = f"http://{domain}:{port}"
path = lambda x: urllib.parse.urljoin(host, x)

In [2]:
import requests

user_id = '70868297-ced7-46d3-8a7a-4ca528e49009'
name = 'hugo-first-avatar'
r = requests.post(path(f'avatars/{user_id}/{name}'))
r

<Response [409]>

In [2]:
pricing_requests = pd.read_csv("pricing_requests.csv")

In [4]:
list_cities = ["amsterdam", "copenhagen", "madrid", "paris", "rome", "sofia", "valletta", "vienna", "vilnius"]
list_date = [k for k in range(45)]
list_languages = ["austrian", "belgian", "bulgarian", "croatian", "cypriot", "czech", "danish", "dutch",
                  "estonian", "finnish", "french", "german", "greek", "hungarian", "irish", "italian",
                  "latvian", "lithuanian", "luxembourgish", "maltese", "polish", "portuguese", "romanian",
                  "slovakian", "slovene", "spanish", "swedish"]

In [5]:
def CreateRequestParams(nb_avatars, nb_requests):
    list_params = []
    for i in range(nb_avatars):
        num_avatar = rd.randint(0, 100000)
        name = "AvatarNumero"+str(num_avatar)
        date = rd.choice(list_date)
        r = requests.post(path(f"avatars/{user_id}/{name}"))
        for j in range(nb_requests):
            language = rd.choice(list_languages)
            city = rd.choice(list_cities)
            mobile = rd.randint(0,1)
            params = {"avatar_name": name,
                       "language": language,
                       "city": city,
                       "date": date,
                       "mobile": mobile,
                    }
            if params not in list_params:
                list_params.append(params)
    return(list_params)

In [6]:
def MakeRequests(list_params):
    pricing_requests = []
    for params in list_params:
        req = requests.get(path(f"pricing/{user_id}"), params=params)
        pricing_requests.append(
        pd.DataFrame(req.json()['prices']).assign(**req.json()['request']))
    pricing_requests = pd.concat(pricing_requests)
    return(pricing_requests)

def MergeRequestsResults(Big_df, New_df):
    big_df = Big_df.copy()
    new_df = New_df.copy()
    new_df["request_id"] = new_df["city"] + new_df["language"] + new_df["avatar_id"].astype(str)
    new_df["request_number"] = np.NaN
    list_request_id = new_df.request_id.unique()
    for request_id in list_request_id:
        temp = big_df[big_df["request_id"] == request_id]
        if len(temp) == 0:
            req_nb = 1
        else:
            req_nb = max(temp.request_number) + 1
        new_df.loc[new_df["request_id"] == request_id, "request_number"] = req_nb
    res = pd.concat([big_df, new_df], ignore_index = True)
    return(res)

In [ ]:
list_params = CreateRequestParams(nb_avatars=10, nb_requests=10)
New_df = MakeRequests(list_params)
pricing_requests = MergeRequestsResults(pricing_requests, New_df)

In [6]:
pricing_requests.to_csv('pricing_requests.csv', index=False)

In [33]:
pricing_requests

,hotel_id,price,stock,city,date,language,mobile,avatar_id,request_id,request_number
0,300,145,1,copenhagen,2,hungarian,0,28434,copenhagen2hungarian28434,1.0
1,855,311,1,copenhagen,2,hungarian,0,28434,copenhagen2hungarian28434,1.0
2,757,151,1,copenhagen,2,hungarian,0,28434,copenhagen2hungarian28434,1.0
3,805,202,2,copenhagen,2,hungarian,0,28434,copenhagen2hungarian28434,1.0
4,930,120,0,copenhagen,2,hungarian,0,28434,copenhagen2hungarian28434,1.0
...,...,...,...,...,...,...,...,...,...,...
163000,720,280,130,copenhagen,31,croatian,0,76537,copenhagen31croatian76537,1.0
163001,994,164,68,copenhagen,31,croatian,0,76537,copenhagen31croatian76537,1.0
163002,744,173,60,copenhagen,31,croatian,0,76537,copenhagen31croatian76537,1.0
163003,124,277,128,copenhagen,31,croatian,0,76537,copenhagen31croatian76537,1.0
